In [13]:
import pandas as pd

df = pd.read_csv("../data/Social_Network_Ads.csv")

df.sample(5)

,User ID,Gender,Age,EstimatedSalary,Purchased
376,15596984,Female,46,74000,0
75,15651983,Male,34,112000,1
65,15718071,Male,24,58000,0
388,15672330,Male,47,34000,1
363,15768293,Female,42,79000,0


In [14]:
print(df.dtypes)

User ID            int64
Gender               str
Age                int64
EstimatedSalary    int64
Purchased          int64
dtype: object


In [15]:
df['Age'] = df['Age'].astype('category')
df['Gender'] = df['Gender'].astype('category')

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df[['EstimatedSalary','Age']]
y = df['Purchased']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 42
)

model = LogisticRegression()
model.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [17]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(y_train.values[:5],y_train_pred[:5])
print(y_test.values[:5],y_test_pred[:5])


[0 0 1 1 1] [0 0 0 1 0]
[0 1 0 1 0] [0 1 0 1 0]


In [ ]:
from sklearn.metrics import confusion_matrix
print("confusion matrix on y_train and y_train_pred")
print(confusion_matrix(y_train, y_train_pred))
print("confusion matrix on y_test and y_text_pred")
print(confusion_matrix(y_test, y_test_pred))

confusion matrix on y_train and y_train_pred
[[169  15]
 [ 32  64]]
confusion matrix on y_test and y_text_pred
[[71  2]
 [16 31]]


In [20]:
from sklearn.metrics import classification_report
print("metrics on y_train and y_train_pred")
print(classification_report(y_train, y_train_pred))
print("metrics on y_test and y_test_pred")
print(classification_report(y_test, y_test_pred))

metrics on y_train and y_train_pred
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       184
           1       0.81      0.67      0.73        96

    accuracy                           0.83       280
   macro avg       0.83      0.79      0.80       280
weighted avg       0.83      0.83      0.83       280

metrics on y_test and y_test_pred
              precision    recall  f1-score   support

           0       0.82      0.97      0.89        73
           1       0.94      0.66      0.78        47

    accuracy                           0.85       120
   macro avg       0.88      0.82      0.83       120
weighted avg       0.86      0.85      0.84       120



In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd

df = pd.read_csv("../data/Social_Network_Ads.csv")

X = df[['EstimatedSalary','Gender', 'Age']]
y = df['Purchased']
categorical_cols = ['Gender', 'Age']
numerical_cols = ['EstimatedSalary']

#--- PROCESS 1: None Normalized ---
# Only transforms categories to numbers; keeps numerical values as they are.
preprocessor_simple = ColumnTransformer(
    transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_cols)
    ],
    remainder='passthrough' # Keeps numerical columns unchanged
)
model_none_normalized = Pipeline(steps=[
    ('preprocessor', preprocessor_simple),
    ('lr', LogisticRegression())
])

# --- PROCESS 2: Normalized Process ---
# Transforms categories AND scales numerical features to a similar range (Mean=0, Std=1).
preprocessor_scaled = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_cols ),
        ('num', StandardScaler(), numerical_cols)
    ]
)
model_normalized = Pipeline(steps=[
    ('preprocessor', preprocessor_scaled),
    ('lr', LogisticRegression())
])
    
# For Target Normalization
model_with_norm = TransformedTargetRegressor(
    regressor=model_normalized,
    transformer=StandardScaler() 
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 42
)

model_none_normalized.fit(X_train, y_train)
model_normalized.fit(X_train, y_train)


/Users/hinadome/code/bootcamp_macaw_04_26/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('lr', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers conta

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_train_pred_none_normalized = model_none_normalized.predict(X_train)
y_test_pred_none_normalized = model_none_normalized.predict(X_test)

y_train_pred_normalized = model_normalized.predict(X_train)
y_test_pred_normalized = model_normalized.predict(X_test)

print("====none-normalized====")
print("confusion matrix on y_train and y_train_pred")
print(confusion_matrix(y_train, y_train_pred_none_normalized))
print("confusion matrix on y_train and y_train_pred")
print(confusion_matrix(y_test, y_test_pred_none_normalized))
print("metrics on y_train and y_train_pred")
print(classification_report(y_train, y_train_pred_none_normalized))
print("metrics on y_test and y_test_pred")
print(classification_report(y_test, y_test_pred_none_normalized))
print("====normalized====")
print("confusion matrix on y_train and y_train_pred")
print(confusion_matrix(y_train, y_train_pred_normalized))
print("confusion matrix on y_train and y_train_pred")
print(confusion_matrix(y_test, y_test_pred_normalized))
print("metrics on y_train and y_train_pred")
print(classification_report(y_train, y_train_pred_normalized))
print("metrics on y_test and y_test_pred")
print(classification_report(y_test, y_test_pred_normalized))

====none-normalized====
confusion matrix on y_train and y_train_pred
[[173  11]
 [ 38  58]]
confusion matrix on y_train and y_train_pred
[[72  1]
 [24 23]]
metrics on y_train and y_train_pred
              precision    recall  f1-score   support

           0       0.82      0.94      0.88       184
           1       0.84      0.60      0.70        96

    accuracy                           0.82       280
   macro avg       0.83      0.77      0.79       280
weighted avg       0.83      0.82      0.82       280

metrics on y_test and y_test_pred
              precision    recall  f1-score   support

           0       0.75      0.99      0.85        73
           1       0.96      0.49      0.65        47

    accuracy                           0.79       120
   macro avg       0.85      0.74      0.75       120
weighted avg       0.83      0.79      0.77       120

====normalized====
confusion matrix on y_train and y_train_pred
[[173  11]
 [ 31  65]]
confusion matrix on y_train and y

/Users/hinadome/code/bootcamp_macaw_04_26/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/hinadome/code/bootcamp_macaw_04_26/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


1. true positive (TP): model correctly predicts positive class
2. true negative (TN): model correctly predicts negative class
3. false positive (FP): model incorrectly predicts positive class (type I error)
4. false negative (FN): model incorrectly predicts negative class (type II error)

# Precision and Recall
- Precision: measures the accuracy of positive predictions
- Recall: measures ability to identify all relevant positive instances

In [24]:
tp = y_train[(y_train == 1) & (y_train_pred == 1)].count()
tn = y_train[(y_train == 0) & (y_train_pred == 0)].count()
fp = y_train[(y_train == 0) & (y_train_pred == 1)].count()
fn = y_train[(y_train == 1) & (y_train_pred == 0)].count()

print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

True Positives: 64
True Negatives: 169
False Positives: 15
False Negatives: 32


In [25]:
example_pred = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1]
example_true = [0, 1, 0, 0, 1, 0, 1, 1, 1, 1]

In [ ]:
# 1 side of Precision
pred_1 = 5
pred_1_matching_true_1 = 4
precision = 4 / 5
print(precision)

0.8


In [ ]:
# 1 side of Recall
pred_1_matching_true_1 = 4
true_1 = 6
recall = 4 / 6
print(recall) 

0.6666666666666666


In [34]:
# 0 side of Precision
pred_0 = 5
pred_0_matching_true_0 = 3
precision = 3 / 5
print(precision)

0.6


In [35]:
# 0 side of Recall
pred_0_matching_true_0 = 3
true_0 = 4
recall = 3 / 4
print(recall)

0.75


In [33]:
from sklearn.metrics import classification_report
print(classification_report(example_true, example_pred))

              precision    recall  f1-score   support

           0       0.60      0.75      0.67         4
           1       0.80      0.67      0.73         6

    accuracy                           0.70        10
   macro avg       0.70      0.71      0.70        10
weighted avg       0.72      0.70      0.70        10

